In [1]:
import pathlib
base_path = pathlib.Path("../")

In [2]:
import torch
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import datetime
import sys
sys.path.append(str(base_path))
from src.models.v1.model import AudioAutoEncoder
from src.models.v1.training import train_autoencoder
from src.dataloaders.v1.dataloader import AudioDB
from src.utils.utils import plot_loss_function

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [3]:
data_path = base_path / 'audio/'
results_path = base_path / 'results/'
models_path = base_path / 'models/'

In [4]:
# Checking if gpu available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Model Set up

In [5]:
autoencoder = AudioAutoEncoder(2, 32, 400)

#Initialize indepdent optimizer for both networks
learning_rate = .001
weight_decay = 1e-5
optimizer = torch.optim.Adam(autoencoder.parameters(),
                             lr=learning_rate, 
                             weight_decay=weight_decay)
lbmda = 0.5

num_epochs = 30

In [6]:
model_timestamp = datetime.datetime.today().strftime("%d-%m-%Y_%H-%M")
os.mkdir(models_path / model_timestamp)
model_name = f"{model_timestamp}/model"
model_hyperparams_filename = models_path / model_timestamp / "hyperparams.json"

print(model_name)

09-12-2022_21-53/model


In [7]:
assert pathlib.Path(models_path / model_timestamp).exists()

In [8]:
# Save hyperparms
params_dict = {
    "learning_rate": learning_rate,
    "weight_decay": weight_decay,
    "optimizer": str(optimizer.__class__),
    "lambda": lbmda,
    "target_num_epochs": num_epochs
}

with open(model_hyperparams_filename, "w") as f:
    json.dump(params_dict, f, indent=4)

# DataLoader

In [9]:
# DataLoader
tr = transforms.Compose([
        transforms.ToTensor(), 
        ])

data_filename = data_path / "train_data.csv"

batch_size = 60
train_audios_db = AudioDB(data_filename,
                   data_path,
                   tr)

num_workers = 8

train_loader = torch.utils.data.DataLoader(dataset=train_audios_db,
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           num_workers = num_workers)

# Training

In [ ]:
model_filename, model_loss_filename, losses_list = train_autoencoder(autoencoder, 
                                                                     train_loader, 
                                                                     optimizer,
                                                                     model_name, 
                                                                     models_path, 
                                                                     num_epochs,
                                                                     device, 
                                                                     lbmda)

# Plot training loss

In [ ]:
plot_loss_function(losses_list, "Training Loss")